In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D,Dense,Embedding,Flatten,Dropout,MaxPooling1D
import nltk
from keras.preprocessing import sequence

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

Used sentiraama dataset available at https://ltrc.iiit.ac.in/showfile.php?filename=downloads/sentiraama/ 

In [3]:
path = '../../data/sentiraama/Song_Lyrics/'

In [4]:
song_list=!find $path -type f -name *.txt;

In [38]:
song_list[:5],song_list[-5:]

(['../../data/sentiraama/Song_Lyrics//positive/song116.txt',
  '../../data/sentiraama/Song_Lyrics//positive/song102.txt',
  '../../data/sentiraama/Song_Lyrics//positive/song49.txt',
  '../../data/sentiraama/Song_Lyrics//positive/song61.txt',
  '../../data/sentiraama/Song_Lyrics//positive/song248.txt'],
 ['../../data/sentiraama/Song_Lyrics//negative/song319.txt',
  '../../data/sentiraama/Song_Lyrics//negative/song325.txt',
  '../../data/sentiraama/Song_Lyrics//negative/song93.txt',
  '../../data/sentiraama/Song_Lyrics//negative/song331.txt',
  '../../data/sentiraama/Song_Lyrics//negative/song44.txt'])

In [7]:
songs=[]
sentences=[];
label=[];

Tokenizing words using nltk. Can also be done using native split() func

In [8]:
for song in song_list:
    aa = ' '.join(open(song).readlines());
    songs=np.append(songs,nltk.word_tokenize(aa))
    sentences.append(nltk.word_tokenize(aa))
    label.append(0 if song.split('/')[-2] == 'negative' else 1)
#     songs.append(np.asarray(nltk.word_tokenize(aa)));

In [9]:
len(label)

337

In [10]:
len(sentences[0])

158

In [11]:
songs.shape

(50060,)

In [ ]:
Let's find the number of unique words'

In [12]:
np.unique(songs).shape

(16057,)

In [13]:
uniq=np.unique(songs,return_index=True,return_counts=True)

In [14]:
aa=np.array(uniq).T

In [15]:
temp=sorted(aa,key=lambda x : int(x[2]), reverse=True)

In [16]:
uniqueElements=[int(aa[1]) for aa in temp]

In [17]:
len(uniqueElements)

16057

In [18]:
vocab=songs[uniqueElements]

In [19]:
vocab

array(['నీ', 'నా', 'ఓ', ..., 'హ…', 'ైవె', '…'], dtype='<U32')

In [20]:
len(vocab)

16057

In [21]:
vocab2idx={o:i for i,o in enumerate(vocab)}

In [39]:
#vocab2idx

In [23]:
idx2vocab={i:o for i,o in enumerate(vocab)}

In [24]:
combined = list(zip(sentences, label))
np.random.shuffle(combined)

sentences[:], label[:] = zip(*combined)

In [25]:
vocab_size=8000;

In [26]:
lens=list(map(lambda x:len(x),sentences))

In [27]:
max(lens),np.min(lens)

(656, 23)

In [28]:
np.argmin(lens)

286

In [29]:
sentences[np.argmin(lens)],label[np.argmin(lens)]

(['ఎవ్వరికి',
  'ఎవ్వరిని',
  'జంటగ',
  'అనుకుంటాడో',
  'ఆఖరికి',
  'వాల్లనే',
  'ఓ',
  'చోట',
  'కలిపేస్తాడు',
  'మనసా',
  'మల్లి',
  'మల్లి',
  'చూసా',
  'గిల్లి',
  'గిల్లి',
  'చూసా',
  'జరిగింది',
  'నమ్మేసా',
  'జతగ',
  'నాతో',
  'నిన్నే',
  'చూసా',
  'నీతో',
  'నన్నే',
  'చూసా',
  'నను',
  'నీకు',
  'వదిలేసా',
  'పై',
  'లోకంలో',
  'వాడు',
  'యెపుడో',
  'ముడి',
  'వేసాడు',
  'విడిపోదే',
  'విడిపోదె',
  'తను',
  'పాల',
  'వెల్లంట',
  'నువు',
  'వాన',
  'జల్లంట',
  'మీలోన',
  'ఈ',
  'ప్రేమ',
  'తీరదమ్',
  'తీరదమ్',
  'తను',
  'కంటి',
  'పాపంట',
  'నువు',
  'కంటి',
  'రెప్పంట',
  'విడదీయలేమంట',
  'ఎవరం',
  'ఎవరం',
  'మనసా',
  'మల్లి',
  'మల్లి',
  'చూసా',
  'గిల్లి',
  'గిల్లి',
  'చూసా',
  'నూరేల్ల',
  'మన',
  'ఆశా',
  'జతగ',
  'నాతో',
  'నిన్నే',
  'చూసా',
  'నీతో',
  'నన్నే',
  'చూసా',
  'నీ',
  'వెంట',
  'అడుగేసా',
  'తీయనైన',
  'చీకటిని',
  'తలుచుకునే',
  'వేకువలో',
  'హాయి',
  'మల్లె',
  'తీగలతో',
  'వేచి',
  'వున్న',
  'వాకిలులూ',
  'నింగి',
  'నేల',
  'గాలి',
  'నీరు',
  'న

In [30]:
seq_len = 500

In [31]:
sentences2idx =[[vocab2idx[i] for i in seq] for seq in sentences]

In [32]:
sentences2idx = [[i if i<vocab_size else vocab_size-1 for i in seq] for seq in sentences2idx]

In [33]:
trn=sentences2idx[:250]
test=sentences2idx[250:]
trn_label=label[:250]
test_label=label[250:]

In [34]:
trn = sequence.pad_sequences(trn,maxlen=seq_len,value=0);
test = sequence.pad_sequences(test,maxlen=seq_len,value=0);

In [35]:
model = Sequential()
model.add(Embedding(vocab_size,32,input_length=seq_len))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           256000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,856,201
Trainable params: 1,856,201
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(trn,trn_label,epochs=3,batch_size=32,validation_data=(test,test_label))

Train on 250 samples, validate on 87 samples
Epoch 1/3
250/250 [==============================] - 1s 3ms/step - loss: 0.6753 - acc: 0.5840 - val_loss: 0.5995 - val_acc: 0.6782
Epoch 2/3
250/250 [==============================] - 0s 1ms/step - loss: 0.6079 - acc: 0.6880 - val_loss: 0.5855 - val_acc: 0.6897
Epoch 3/3
250/250 [==============================] - 0s 1ms/step - loss: 0.5728 - acc: 0.6840 - val_loss: 0.5752 - val_acc: 0.6782
